# ファーマコインフォマティクスシミュレーション特論

小寺正明

# 第2回　適用範囲

機械学習モデルの性能を考えるにあたって「過剰適合」を取り扱いましたが、「適用範囲」（Applicability Domain, AD）も重要な視点です。

# １次元データ

適用範囲とは何か、まずは１次元で考えてみましょう。乱数を用いて、次のようなデータを用意します。

In [ ]:
import numpy as np

X = np.concatenate([np.random.normal(20, 10, (10, 1)), np.random.normal(80, 10, (10, 1))])

具体的には、次のような数字の集合です。

In [ ]:
X

データの分布をヒストグラムとして表すと、次のようになります。

In [ ]:
import matplotlib.pyplot as plt

plt.hist(X)
plt.show()

ヒストグラムの下に、一次元散布図としてデータをプロットしてみましょう。

In [ ]:
plt.scatter(X, [-0.5 for x in X], alpha=0.5)
plt.hist(X)
plt.show()

以上のデータを用いて、適用範囲について考えてみたいと思います。

# K近傍法（k-NearestNeighbors, k-NN）

おそらく最もわかりやすい K近傍法 を使ってみます。

In [ ]:
from sklearn.neighbors import NearestNeighbors

n_neighbors = 3 # 自分自身を含めて k 番目に近い点までを「近傍」とみなす
model = NearestNeighbors(n_neighbors=n_neighbors) # モデルの立ち上げ
model.fit(X) # X のデータ構造を学習

学習の結果をもとに、X のデータを距離データとインデックスデータに変換します。

In [ ]:
distances, indices = model.kneighbors(X)

インデックスデータには、自分自身を含めて k 番目に近い点までのインデックスが蓄えられます。

In [ ]:
indices

距離データには、（インデックスデータに対応する）自分を含めて k 番目に近い点までの距離の情報が蓄えられます。

In [ ]:
distances

上の計算結果から、n_neighbors 番目に近い点までの距離をリストにしてみましょう。

In [ ]:
distances[:, n_neighbors - 1]

それを小さい順にソートします。

In [ ]:
sorted(distances[:, n_neighbors - 1])

ここから、たとえば out = 0.2 として、全データのうち 20% を「外れ値」とみなしたときに、どこを閾値とすれば良いかを考えます。実際にはこの out = 0.2 という値は、0.05 など、もっと小さい値を設定するのが普通なのですが、今回は説明のためにわざと大きい値を設定します。

In [ ]:
out = 0.2
int((len(X) - 1) * (1 - out))

この順位のdistanceを次のように取得して、閾値とします。

In [ ]:
out = 0.2
threshold = sorted(distances[:, n_neighbors - 1])[int((len(X) - 1) * (1 - out))]
threshold

上記の値を閾値として、それよりも大きいdistanceならば外れ値、すなわち「適用範囲外」とみなすわけです。

以上の計算をもとに、適用範囲を可視化しましょう。まず、Xを k neighbor distanceに変換する関数を次のように定義します。

In [ ]:
def transform(x):
    distances, indices = model.kneighbors(x)
    return distances[:, n_neighbors - 1]

Xの定義域を 0 から 100 までとします。

In [ ]:
x_latent = np.linspace(0, 100, 101).reshape(101, 1)

データXと、閾値と、k neighbor distanceとの関係を図示すると次のようになります。

In [ ]:
plt.plot(x_latent, transform(x_latent), label="k neighbor distance")
plt.plot([0, 100], [threshold, threshold], label="threshold")
plt.scatter(X, [-0.5 for x in X], alpha=0.5, label="data")
plt.legend()
plt.show()

k neighbor distance の値が小さい領域が、データ密度の高い領域になります。そして、その値が閾値以下の部分は、十分にデータ密度が高く、何か予測を行った場合に、その予測が信頼できるであろう領域、つまり「適用範囲」であるとみなすわけです。

# One Class SVM (OCSVM)

適用範囲の定義の仕方は他にもたくさんあります。今回はもうひとつだけ、One Class SVM という方法について簡単にお示ししましょう。One Class SVM は名前の通り SVM を応用したもので、外れ値なのかそうじゃないのかをSVMで分類するモデルです。学習は次のように行います。

In [ ]:
from sklearn.svm import OneClassSVM

model = OneClassSVM() # モデルの立ち上げ
model.fit(X) # X のデータ構造を学習

SVMでは、分類予測結果を判断するためのスコアを次のように算出します。

In [ ]:
score = model.decision_function(X)
score

そーっとソートしてみましょう。

In [ ]:
sorted(score)

k近傍法では distance が小さいほど「近傍」である、つまりデータ密度が高いということでしたが、OCSVM（および他のほぼ全ての方法）では、スコアが大きいほどデータ密度が高いという尺度になります。ですので、 out = 0.2 つまりデータの 20% を「外れ値」とみなしたい場合の閾値は次のようにして決定します。

In [ ]:
out = 0.2
int((len(X) - 1) * out)

In [ ]:
out = 0.2
threshold = sorted(score)[int((len(X) - 1) * out)]
threshold

上記の閾値をよりも低いスコアのものを「外れ値」つまり「適用範囲外」とみなすわけです。寿司しましょう。

In [ ]:
plt.plot(x_latent, model.decision_function(x_latent))
plt.plot([0, 100], [threshold, threshold])
plt.scatter(X, [1 for x in X], alpha=0.5)
plt.show()

同じデータでも、k近傍法とOCSVMで適用範囲の領域が若干異なることが分かります。どっちが良いかや、何％を外れ値とみなせば良いかなどは、ケースバイケースで検討してみましょう。

# ２次元データ

それでは、今度は２次元データを描画しながら<s>遊んで</s>学んでみましょう。まずは、２次元データを可視化するための関数を用意しておきます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def heatmap(f, x_min=-12, x_max=12, y_min=-12, y_max=12, h=0.1, 
             axes=[0, 1, 1, 1], epsilon=None,
             drawline=False, cmap=plt.cm.jet):

    def get_Z(f, axis):
        if axis == 1:
            X = []
            for y in np.arange(y_min, y_max, h):
                for x in np.arange(x_min, x_max, h):
                    X.append([x, y])
            X = np.array(X)
            Z = f(X).reshape(
                len(np.arange(x_min, x_max, h)), 
                len(np.arange(y_min, y_max, h))
                )
        else:
            Z = [[f([x, y]) for x, y in zip(xx, yy)] for xx, yy in zip(x_mg,y_mg)]
        return Z

    x_mg, y_mg = np.meshgrid(
        np.arange(x_min, x_max, h), 
        np.arange(y_min, y_max, h)
        )
    if type(f) == list:
        for i, ff in enumerate(f):
            if axes[i] == -1:
                plt.contour(x_mg, y_mg, get_Z(ff, 1), colors='black')
            elif i == 0:
                Z = np.array(get_Z(ff, axes[i]))
            else:
                Z -= np.array(get_Z(ff, axes[i]))
        Z = Z.tolist()
            
    else:
            Z = get_Z(f, axes[0])

    if epsilon:
        Z = np.array(Z)
        Z = np.ma.array(Z, mask=np.abs(Z) <= epsilon)
        cmap.set_bad((1, 1, 1, 1))  # 無効な値に対応する色
        #cmap.set_bad((0, 0, 0, 1))  # 無効な値に対応する色

    plt.axes().set_aspect('equal')
    if drawline:
        plt.contour(x_mg, y_mg, Z, colors='black')
    plt.imshow(Z, origin='lower', extent=[x_min, x_max, y_min, y_max], cmap=cmap)
    plt.colorbar()
    #plt.grid()
    plt.show()

そうすると、たとえば次のような２変数関数を図示できます。これらの２変数関数が、予測すべき「真の現象」であるものとします。


In [ ]:
f1 = lambda x: np.sqrt(np.abs(np.sqrt(np.abs(np.minimum((x[0]/3)**2 + (x[1]/3)**2 - 1, (np.abs(x[0]/3) - 0.95)**2 + (x[1]/3 - 0.95)**2 - 0.55**2)))))
f2 = lambda x: np.sqrt(x[0]**2 + (x[1] - np.sqrt(np.abs(x[0])))**2)
f3 = lambda x: np.linalg.norm([x[0], x[1]]) - np.cos(6 * np.arctan2(x[0],  x[1]))
f4 = lambda x: np.linalg.norm([x[0], x[1]]) - 2* np.arctan2(x[0],  x[1]) 

func = [f1, f2, f3, f4]
for f in func:
    heatmap(f, drawline=False)

# training data, validation data, test data

ところが、現実に測定可能なのは「真の現象」の中のほんのいくつかの点でしかありません。現実に測定した点が次のような分布を持っていたとしましょう。これを training data とします。

In [ ]:
X_train = np.random.rand(2, 20) * 10

plt.scatter(X_train.T[:, 0], X_train.T[:, 1], label="train")
plt.axes().set_aspect('equal')
plt.xlim([-12, 12])
plt.ylim([-12, 12])
plt.legend()
plt.grid()
plt.show()

これに対して、実際に予測したいデータは、学習用データとは異なる分布を持つ場合があります。このような場合に、適用範囲外の予測結果はどのようになるか見てみましょう。

ちなみに、適用範囲内の予測を「内挿」、適用範囲外の予測を「外挿」と呼ぶことがあります。

In [ ]:
X_valid = np.random.rand(2, 20) * 15 - 5
X_test = np.random.rand(2, 20) * 15 - 10

plt.scatter(X_train.T[:, 0], X_train.T[:, 1], marker='o', label="train")
plt.scatter(X_valid.T[:, 0], X_valid.T[:, 1], marker='x', label="valid")
plt.scatter(X_test.T[:, 0], X_test.T[:, 1], marker='s', label="test")
plt.axes().set_aspect('equal')
plt.xlim([-12, 12])
plt.ylim([-12, 12])
plt.legend()
plt.grid()
plt.show()

# データ密度を定義するさまざまな手法

このようなとき、学習に用いたデータの密度の濃い部分だけを信用するということをよく行います。その「学習データの密度」を計算する手法として、

- K-Nearest Neighbors (KNN)
- One-Class SVM (OCSVM)
- Isolation Forest (IF)
- Local Outlier Factor (LOF)
- Elliptic Envelope (EE)

などの手法があります。試しに使ってみましょう。

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope

class KNN:
    def __init__(self, n_neighbors=5, outlier_fraction=0.1, algorithm='ball_tree'):
        self.n_neighbors = n_neighbors
        self.outlier_fraction = outlier_fraction
        self.model = False
        self.algorithm = algorithm
        self.distances = False
        self.indices = False
        self.threshold = False
        self.len_data = False
        
    def fit(self, X):
        self.len_data = len(X)
        self.model = NearestNeighbors(
            n_neighbors=self.n_neighbors, 
            algorithm=self.algorithm 
            ).fit(X)
        self.distances, self.indices = self.model.kneighbors(X)
        self.threshold = sorted(
            self.distances[:, self.n_neighbors - 1]
            )[int((self.len_data - 1) * (1 - self.outlier_fraction))]

    def transform(self, x):
        self.distances, self.indices = self.model.kneighbors(x)
        return self.distances[:, self.n_neighbors - 1]

    def transform_bin(self, x):
        self.transform(x)
        self.Z = self.distances[:, self.n_neighbors - 1]
        return np.where(self.Z >= self.threshold, 0, 1)

class AVDetector:
    def __init__(self, model=OneClassSVM(), outlier_fraction=0.1, ):
        self.model = model
        self.outlier_fraction = outlier_fraction
        self.threshold = False
        self.len_data = False
    
    def fit(self, X):
        self.len_data = len(X)
        self.model.fit(X)
        self.threshold = sorted(
            self.model.decision_function(X)
            )[int((self.len_data - 1) * self.outlier_fraction)]

    def transform(self, x):
        return self.model.decision_function(x)

    def transform_bin(self, x):
        self.Z = self.model.decision_function(x)
        return np.where(self.Z >= self.threshold, 1, 0)

先ほどのデータ分布に対して、各手法がどのような適用範囲を定義するか見てみましょう。太線の内側が「適用範囲」とみなされることになります。どの手法が最も良いかと言われても、ケースバイケースとしか言いようがないかも知れません。

In [ ]:
avd_dict = {}
for avd_name, avd in [
            ["KNN", KNN()], 
            ["OneClassSVM", AVDetector(model=OneClassSVM())], 
            ["IsolationForest", AVDetector(model=IsolationForest())],
            ["LocalOutlierFactor", AVDetector(model=LocalOutlierFactor(novelty=True))],
            ["EllipticEnvelope", AVDetector(model=EllipticEnvelope())]
]:
    avd.fit(X_train.T)
    avd_dict[avd_name] = avd
    plt.title(avd_name)
    plt.scatter(X_train.T[:, 0], X_train.T[:, 1])
    heatmap([avd.transform, avd.transform_bin], axes=[1, -1], drawline=False)

機械学習を用いた予測手法にもさまざまな「クセ」があるように、適用範囲を極める手法にもさまざまな「クセ」があることがお分かりいただけるのではないかと思います。

# 機械学習による予測と、適用範囲との関係

それでは、さまざまな機械学習手法に対して、典型的な予測結果の傾向と、適用範囲との関係を見ていきましょう。その前に、結果表示用の関数を定義します。

In [ ]:
from sklearn.metrics import mean_squared_error, balanced_accuracy_score

def compare_model(func, predict, cmap=plt.cm.jet, epsilon=0.1, metrics=mean_squared_error, alpha=1.0):
    metrics_train = metrics(Y_train, predict(X_train.T))
    metrics_valid = metrics(Y_valid, predict(X_valid.T))
    metrics_test = metrics(Y_test, predict(X_test.T))

    plt.title("True function")
    plt.scatter(X_train.T[:, 0], X_train.T[:, 1], marker='o', alpha=alpha)
    plt.scatter(X_valid.T[:, 0], X_valid.T[:, 1], marker='^', alpha=alpha)
    plt.scatter(X_test.T[:, 0], X_test.T[:, 1], marker='s', alpha=alpha)
    heatmap([func, avd.transform_bin], axes=[0, -1], drawline=False, cmap=cmap)

    plt.title("Predicted function")
    plt.scatter(X_train.T[:, 0], X_train.T[:, 1], marker='o', alpha=alpha)
    plt.scatter(X_valid.T[:, 0], X_valid.T[:, 1], marker='^', alpha=alpha)
    plt.scatter(X_test.T[:, 0], X_test.T[:, 1], marker='s', alpha=alpha)
    heatmap([predict, avd.transform_bin], axes=[1, -1], drawline=False, cmap=cmap)

    plt.title("train={0:.2f}, test={1:.2f}, valid={2:.2f}".format(metrics_train, metrics_test, metrics_valid))
    plt.scatter(X_train.T[:, 0], X_train.T[:, 1], marker='o', alpha=alpha)
    plt.scatter(X_valid.T[:, 0], X_valid.T[:, 1], marker='^', alpha=alpha)
    plt.scatter(X_test.T[:, 0], X_test.T[:, 1], marker='s', alpha=alpha)
    heatmap([predict, func, avd.transform_bin], axes=[1, 0, -1], drawline=False, cmap=cmap, epsilon=epsilon)

適用範囲を定義する手法を選択します。

In [ ]:
avd = avd_dict["OneClassSVM"]

予測したい２変数関数を選択します。

In [ ]:
f = f1
Y_train = f(X_train)
Y_valid = f(X_valid)
Y_test = f(X_test)

それでは、順次、いろんな機械学習手法を使ってみましょう。なお、ここではハイパーパラメーターチューニングを省略します。

３つの図が現れますが、上から順に、

- 真の現象（２変数関数）
- 予測値
- 両者のズレ

を表します。ズレの小さい部分は白色になります。

In [ ]:
from sklearn.svm import SVR

model = SVR()
model.fit(X_train.T, Y_train)
compare_model(f, model.predict)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train.T, Y_train)
compare_model(f, model.predict)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor()
model.fit(X_train.T, Y_train)
compare_model(f, model.predict)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()
model.fit(X_train.T, Y_train)
compare_model(f, model.predict)

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=[100]*10)
model.fit(X_train.T, Y_train)
compare_model(f, model.predict)

各々の学習モデルの「くせ」、そしてそれと適用範囲との関係がなんとなく把握できたのではないでしょうか。

# Optuna を用いたハイパーパラメーターチューニング

最後に、Optuna を用いたハイパーパラメーターチューニングを行いながら、適用範囲との関係を見てみたいと思います。まずは Optuna のインストールから。

In [ ]:
!pip install optuna

# データと適用範囲

そして、データの数も適当に増やしてみましょう。train, validation, test で分布域が異なるようにわざとデータを作ったところがポイントです。

In [ ]:
X_train = np.random.rand(2, 100) * 10
X_valid = np.random.rand(2, 100) * 12 - 4
X_test = np.random.rand(2, 100) * 15 - 10

plt.scatter(X_train.T[:, 0], X_train.T[:, 1], marker='o', label="train")
plt.scatter(X_valid.T[:, 0], X_valid.T[:, 1], marker='x', label="valid")
plt.scatter(X_test.T[:, 0], X_test.T[:, 1], marker='s', label="test")
plt.axes().set_aspect('equal')
plt.xlim([-12, 12])
plt.ylim([-12, 12])
plt.legend()
plt.grid()
plt.show()

さまざまな手法で適用範囲を定義すると次のようになります。

In [ ]:
avd_dict = {}
for avd_name, avd in [
            ["KNN", KNN()], 
            ["OneClassSVM", AVDetector(model=OneClassSVM())], 
            ["IsolationForest", AVDetector(model=IsolationForest())],
            ["LocalOutlierFactor", AVDetector(model=LocalOutlierFactor(novelty=True))],
            ["EllipticEnvelope", AVDetector(model=EllipticEnvelope())]
]:
    avd.fit(np.hstack([X_train, X_valid]).T)
    avd_dict[avd_name] = avd
    plt.title(avd_name)
    plt.scatter(np.hstack([X_train, X_valid]).T[:, 0], np.hstack([X_train, X_valid]).T[:, 1])
    heatmap([avd.transform, avd.transform_bin], axes=[1, -1], drawline=False)

# SVMによる予測（チューニングなし）

適用範囲を定義する手法を選択します。

In [ ]:
avd = avd_dict["OneClassSVM"]

予測したい２変数関数を選択します。

In [ ]:
f = f1
Y_train = f(X_train)
Y_valid = f(X_valid)
Y_test = f(X_test)

In [ ]:
from sklearn.svm import SVR

model = SVR()
model.fit(X_train.T, Y_train)
compare_model(f, model.predict, alpha=0.1)

# SVMによる予測（チューニングあり）

次のようなクラスを作って、SVMをOptunaでハイパーパラメーターチューニングします。

In [ ]:
import optuna
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

class OptunaSVR:
    def __init__(self, X_train, X_valid, y_train, y_valid):
        self.X_train = X_train
        self.X_valid = X_valid
        self.y_train = y_train
        self.y_valid = y_valid
        self.best_score = None
        self.best_model = None

    def __call__(self, trial):
        params = {
            "C": trial.suggest_float("C", 1e-10, 1e10, log=True),
            "gamma" : trial.suggest_float("gamma", 1e-10, 1e10, log=True),
            "epsilon" : trial.suggest_float("epsilon", 1e-10, 1e10, log=True),
            "kernel" : "rbf",
            "max_iter": 530000,
        }
        model = SVR(**params)
        model.fit(self.X_train, self.y_train)
        score = mean_squared_error(model.predict(self.X_valid), self.y_valid)

        if self.best_score is None or self.best_score > score:
            self.best_score = score
            self.best_model = model

        return score

目的変数を次のようにセットします。

In [ ]:
objective = OptunaSVR(X_train.T, X_valid.T, Y_train, Y_valid)

次のようにすることで、チューニングの履歴を残すことができます。履歴を残しておくと、中断したチューニングを途中から再開することが可能になります。

In [ ]:
study_name = "optuna_svr8"
strage_name = "optuna_svr8.sql"
study = optuna.create_study(
    study_name = study_name,
    storage='sqlite:///' + strage_name, 
    load_if_exists=True,
    direction="minimize")

では、チューニング開始です。


In [ ]:
timeout = 100
n_trials = 100
show_progress_bar = True
study.optimize(
    objective, timeout=timeout, n_trials=n_trials, show_progress_bar=show_progress_bar
)

ベストモデルとして選ばれたのはこのモデルです。

In [ ]:
objective.best_model

ベストモデルによる予測結果を可視化して、適用範囲との関係を見てみましょう。

In [ ]:
compare_model(f, objective.best_model.predict, alpha=0.1)

チューニングによって改善したかどうかは微妙な気がしますが...（改善どころか改悪する場合もあります）

チューニングの履歴は次のように確認できます。

In [ ]:
study.trials_dataframe()

Optunaでは、ハイパーパラメーターの重要度を算出するオプションも提供しています。

In [ ]:
optuna.visualization.plot_param_importances(
    study=study,
).show()

# MLPによる予測（チューニングあり）

基本的な方法論は上記で既に述べた通りですが、ケーススタディとしてMLPによる予測もやってみましょう。たとえば次のように最適化クラスを定義します。

In [ ]:
import optuna
from sklearn.neural_network import MLPRegressor

# optuna.logging.set_verbosity(optuna.logging.WARN)

class OptunaMLPRegressor:
    def __init__(self, X_train, X_valid, y_train, y_valid):
        self.X_train = X_train
        self.X_valid = X_valid
        self.y_train = y_train
        self.y_valid = y_valid
        self.best_score = None
        self.best_model = None

    def __call__(self, trial):
        n = trial.suggest_int("n", 3, 10) # 層の深さ
        h1 = trial.suggest_int("h1", 2, 128) # 入力層のニューロン数
        h2 = trial.suggest_int("h2", 2, 128) # 中間層のニューロン数
        h3 = trial.suggest_int("h3", 2, 128) # 出力層のニューロン数
        hidden_layer_sizes = []
        for h in range(n):
            if h == 0:
                hidden_layer_sizes.append(h1)
            elif h == n - 1:
                hidden_layer_sizes.append(h3)
            else:
                hidden_layer_sizes.append(h2)

        learning_rate = trial.suggest_categorical( # 学習率のスケジューリング
            "learning_rate", ["constant", "invscaling", "adaptive"]
        )
        learning_rate_init = trial.suggest_float( # 初期の学習率
            "learning_rate_init", 0.0001, 0.01, log=True
        )
        activation = trial.suggest_categorical( # 活性化関数
            "activation", ["logistic", "tanh", "relu"]
        )
        # バッチサイズ
        batch_size = trial.suggest_int("batch_size", 2, self.X_train.shape[0])

        params = {
                "max_iter": 2000, # 計算繰り返し回数の上限
                "random_state": 0,
                "hidden_layer_sizes": hidden_layer_sizes,
                "learning_rate": learning_rate,
                "learning_rate_init": learning_rate_init,
                "activation": activation,
                "batch_size": batch_size,
                "early_stopping": True,
        }
        model = MLPRegressor(**params) # 予測モデルの立ち上げ
        model.fit(self.X_train, self.y_train) # training data を用いた学習
        # validation data を用いたスコア算出
        score = mean_squared_error(model.predict(self.X_valid), self.y_valid, squared=False)

        # ベストスコアが出たら保存
        if self.best_score is None or self.best_score > score:
            self.best_score = score
            self.best_model = model

        return score

目的変数を次のようにセットします。

In [ ]:
objective = OptunaMLPRegressor(X_train.T, X_valid.T, Y_train, Y_valid)

次のようにすることで、チューニングの履歴を残すことができます。履歴を残しておくと、中断したチューニングを途中から再開することが可能になります。

In [ ]:
study_name = "optuna_mlp3"
strage_name = "optuna_mlp3.sql"
study = optuna.create_study(
    study_name = study_name,
    storage='sqlite:///' + strage_name, 
    load_if_exists=True,
    direction="minimize")

では、チューニング開始です。

In [ ]:
timeout = 100
n_trials = 100
show_progress_bar = True
study.optimize(
    objective, timeout=timeout, n_trials=n_trials, #show_progress_bar=show_progress_bar
)

ベストモデルのパラメーターはこんな感じ

In [ ]:
study.best_params

ハイパラチューニングの履歴はこのようになります。

In [ ]:
study.trials_dataframe()

ハイパラの重要度はこのようになります。

In [ ]:
optuna.visualization.plot_param_importances(
    study=study,
).show()

ベストモデルによる予測結果を寿司するとこんな感じ。

In [ ]:
compare_model(f, objective.best_model.predict, alpha=0.1)

# LightGBM

最後に、非深層系で最強クラスと噂される LightGBM を使ってみます。Optunaを用いた普通のチューニングも可能なのですが、次のような特別なインテグレーションもOptunaで提供されています（使い方が異なる部分が多いので、慣れないとちょっと辛い）

In [ ]:
import optuna.integration.lightgbm as lgb

lgb_train = lgb.Dataset(X_train.T, Y_train)
lgb_test = lgb.Dataset(X_valid.T, Y_valid, reference=lgb_train)
lgb_results={}
class OptunaGBM:
    def __init__(self, X_train, X_valid, y_train, y_valid):
        self.X_train = X_train
        self.X_valid = X_valid
        self.y_train = y_train
        self.y_valid = y_valid
        self.best_score = None
        self.best_model = None

    def __call__(self, trial):
        params = {
                'task': 'train',                 
                'boosting_type': 'gbdt',         
                'objective': 'regression',       
                'metric': ['rmse'],             
                'learning_rate': trial.suggest_float('learning_rate', 0.1, 1.0),  
                'num_leaves': trial.suggest_int("num_leaves", 5, 50),
                'tree_learner': trial.suggest_categorical('tree_learner', ["serial", "feature", "data", "voting"]),
                'seed': 530000                     
                }
        model = lgb.train(
                    params=params,                    
                    train_set=lgb_train,              
                    valid_sets=[lgb_train, lgb_test], 
                    valid_names=['Train', 'Test'],    
                    num_boost_round=100,              
                    early_stopping_rounds=50,         
                    evals_result=lgb_results,
                    #verbose_eval=-1,                 
                    )
        score = mean_squared_error(model.predict(self.X_valid), self.y_valid, squared=False)

        if self.best_score is None or self.best_score > score:
            self.best_score = score
            self.best_model = model

        return score

目的変数を次のようにセットします。

In [ ]:
objective = OptunaGBM(X_train.T, X_valid.T, Y_train, Y_valid)

次のようにすることで、チューニングの履歴を残すことができます。履歴を残しておくと、中断したチューニングを途中から再開することが可能になります。

In [ ]:
study_name = "optuna_gbm1"
strage_name = "optuna_gbm1.sql"
study = optuna.create_study(
    study_name = study_name,
    storage='sqlite:///' + strage_name, 
    load_if_exists=True,
    direction="minimize")

では、チューニング開始です。

In [ ]:
timeout = 100
n_trials = 100
show_progress_bar = True
study.optimize(
    objective, timeout=timeout, n_trials=n_trials, #show_progress_bar=show_progress_bar
)

ベストモデルのパラメーターはこんな感じ

In [ ]:
study.best_params

ハイパラチューニングの履歴はこのようになります。

In [ ]:
study.trials_dataframe()

ハイパラの重要度はこのようになります。

In [ ]:
optuna.visualization.plot_param_importances(
    study=study,
).show()

ベストモデルによる予測結果を寿司するとこんな感じ。

In [ ]:
compare_model(f, objective.best_model.predict, alpha=0.1)

# 課題

1. 適用範囲を定義するためのさまざまなモデルの「クセ」を簡単に説明してください。
2. さまざまな予測モデルの「クセ」を簡単に説明してください。
3. 適用範囲の内側と外側に対して、さまざまな予測モデルがどのような挙動を示したか説明してください。
4. 関数 f4 に対して同じ計算を行い、関数 f1 による結果と照らし合わせて、共通点や相違点を説明してください。



**提出方法**：

下記のいずれかの方法で提出してください。

- Google Colaboratory 上で動作させたコードを ikemenmaskot@gmail.com に「共有」

- Jupyter Notebook 上で動作させたコードを ipynb 形式または html 形式にして ikemenmaskot@gmail.com に「メール送信」


# 次回

これまで、創薬と全く関係のない機械学習の話をしてきましたが、第３回はいよいよ RDKit というツールを用いて化学構造の演算の説明をしたいと思います。